  \begin{equation}\label{eq:ndlm}
    \dot{x}_1=\sigma x_2\,,\quad \dot{x}_2=-x_1x_3+rx_1\,,\quad \dot{x}_3=x_1x_2
  \end{equation}
  $x_4=\sigma, x_5=r$

In [51]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
m = 5  # number of trajectories
m1 = 4  # data points per trajectory divided by 2, which means the total data point is m*(2m1)=40
def generate_random_values():
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x3 = random.uniform(0, 1)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    return x1, x2, x3, x4, x5
def calculate_c(x1, x3, x4):
    c = 1 / 2 * x1 ** 2 - x4 * x3
    return c
def generate_random_values_based_on_c(c):
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    x3 = (x1 ** 2 - c) / (2 * x4)
    return x1, x2, x3, x4, x5
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0:
            return vector
        return vector / norm    
    def normalized_system(y, t):
        x1, x2, x3, x4, x5 = y
        f = np.array([x4 * x2, x1 * (x5 - x3), x1 * x2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f    
    t = np.linspace(0, 10, 4)
    forward_trajectories = []
    backward_trajectories = []
    output_directory1 = r'../../../results/Lorenz_System/Lorenz_System_T5_8'
    plt.figure(figsize=(10, 6))
    for initial_condition in initial_conditions:
        forward_trajectory = odeint(normalized_system, initial_condition, t)
        backward_trajectory = odeint(normalized_system, initial_condition, -t)
        forward_trajectories.append(forward_trajectory[:m1])
        backward_trajectories.append(backward_trajectory[:m1])
        for j in range(forward_trajectory.shape[1]):
            plt.plot(t, forward_trajectory[:, j])
            plt.plot(-t, backward_trajectory[:, j]) 
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
    return forward_trajectories, backward_trajectories, t[:m1]
if __name__ == "__main__":
    x1, x2, x3, x4, x5 = generate_random_values()
    c = calculate_c(x1, x3, x4)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(m)]
    forward_trajectories, backward_trajectories, t = generate_data(initial_conditions)    

In [4]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import os
import random
from sklearn.model_selection import train_test_split
output_directory = r'../../../Data/Lorenz_System/Lorenz_System_T5_8'
output_director = r'../../../System/Lorenz_System/Lorenz_System_T5_8'
m = 5 # number of trajectory
m1 = 5 # data point per trajectory divided by 2, which means the total data point is m*(2m1)=40
m2 =4 
def generate_random_values():
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x3 = random.uniform(0, 1)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    return x1, x2, x3, x4, x5
def calculate_c(x1, x3, x4):
    c = 1/2 * x1**2 - x4 * x3
    return c
def generate_random_values_based_on_c(c):
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    x3 = (x1**2 - c) / (2 * x4)
    return x1, x2, x3, x4, x5
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4, x5 = y
        f = np.array([x4 * x2, x1 * (x5 - x3), x1 * x2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    t1 = np.linspace(0, 10, m1) # forward
    t2 = np.linspace(0, 10, m2) # backward
    forward_trajectories = []
    backward_trajectories = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5):")
    for initial_condition in initial_conditions:
        print(initial_condition) 
        forward_trajectory = odeint(normalized_system, initial_condition, t1)
        backward_trajectory = odeint(normalized_system, initial_condition, -t2)
        forward_trajectories.append(forward_trajectory[:m1])
        backward_trajectories.append(backward_trajectory[:m2])
    return forward_trajectories, backward_trajectories, t1[:m1], t2[:m2]
def save_data(forward_trajectories, backward_trajectories, t1, t2):
    num_variables = 5 
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')    
    with open(os.path.join(output_director, '50.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, (forward_data, backward_data) in enumerate(zip(forward_trajectories, backward_trajectories)):
            for j in range(len(t1)):
                forward_row = forward_data[j].tolist() + [r + 1]
                writer.writerow(forward_row)
            for j in range(1, len(t2)):  # Exclude initial conditions for backward trajectory
                backward_row = backward_data[j].tolist() + [r + 1]
                writer.writerow(backward_row)     
    output_directory1 = r'../../../results/Lorenz_System/Lorenz_System_T5_8'
    plt.figure(figsize=(10, 6))
    for i, (forward_sol, backward_sol) in enumerate(zip(forward_trajectories, backward_trajectories)):
        for j in range(forward_sol.shape[1]):
            plt.plot(t1, forward_sol[:, j])
            plt.plot(-t2, backward_sol[:, j])  # Exclude initial conditions for backward trajectory
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None    
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)   
    for traj_points in trajectories.values():
        random.shuffle(traj_points)   
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)    
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point) 
if __name__ == "__main__":
    x1, x2, x3, x4, x5 = generate_random_values()
    c = calculate_c(x1, x3, x4)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(m)]
    forward_trajectories, backward_trajectories, t1, t2 = generate_data(initial_conditions)
    save_data(forward_trajectories, backward_trajectories, t1, t2)
    data = np.genfromtxt(os.path.join(output_director, '50.csv'), delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5):
(1.9417777326623429, 1.07829541911076, 3.2203103650570877, 0.4182485155852582, 1.3099662862970025)
(1.4310418724275031, 1.9547937852666961, 0.49424995577560077, 0.9824584970082691, 0.21187897635361513)
(1.3290677742283998, -1.1503367822907204, 0.11517281769753335, 2.9941980192683544, 1.4847690257059036)
(1.2618020175094586, 1.9089685916013073, 0.1899629348807526, 1.3566426220701393, 0.11339317712141704)
(-1.3335510003979003, -0.5393204063701234, 0.11120114859818239, 3.154812590795018, 0.8551203111059243)


In [5]:
import pandas as pd
import os
input_directory = r'../../../System/Lorenz_System/Lorenz_System_T5_8'
output_directory = r'../../../Data/Lorenz_System/Lorenz_System_T5_8/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/Lorenz_System/Lorenz_System_T5_8/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)